In [1]:
import os
from dotenv import load_dotenv
load_dotenv('.env', verbose=True)

from typing import Literal, TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.graph import StateGraph, END, MessageGraph

In [2]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from typing import Literal, List

@tool
def multiply(num1, num2):
    """ miltiplies two numbers together """

    return num1 * num2

def router(state: List[BaseMessage]) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

model = ChatOpenAI(name='gpt-3.5-turbo')
model_with_tools = model.bind_tools([multiply])
tool_node = ToolNode([multiply])

builder = MessageGraph()
builder.add_node("llm", model_with_tools)
builder.add_node("multiply", tool_node)

builder.set_entry_point('llm')
builder.add_edge("llm", "multiply")
builder.add_conditional_edges("llm", router)
builder.add_edge("multiply", END)

runnable = builder.compile()
runnable.invoke(HumanMessage("what is 123 * 456 * 798 ?"))

[HumanMessage(content='what is 123 * 456 * 798 ?', id='2c423731-25a6-4fde-806b-99551fb5ec0a'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mCOW4yUYXgt96vYYAqMMJzBa', 'function': {'arguments': '{"num1": 123, "num2": 456}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_MtS9p07CzqvWCpgr7l60Qdxr', 'function': {'arguments': '{"num1": 56088, "num2": 798}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 62, 'total_tokens': 116}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-159fe921-1888-4357-b3ea-f48d16cae9b3-0', tool_calls=[{'name': 'multiply', 'args': {'num1': 123, 'num2': 456}, 'id': 'call_mCOW4yUYXgt96vYYAqMMJzBa'}, {'name': 'multiply', 'args': {'num1': 56088, 'num2': 798}, 'id': 'call_MtS9p07CzqvWCpgr7l60Qdxr'}], usage_metadata={'input_tokens': 62, 'output_tokens': 54, 'total_tokens': 116}),
 ToolMessage(conte

In [3]:
from IPython.display import Image, display

display(Image(builder.get_graph().draw_mermaid_png()))


AttributeError: 'MessageGraph' object has no attribute 'get_graph'